<a href="https://colab.research.google.com/github/450586509/Knowledge/blob/master/event_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install transformers

In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as tfs
import warnings

warnings.filterwarnings('ignore')

In [23]:
dev_url = "http://129.204.205.246/event_entity_dev_data.csv"
train_url = "http://129.204.205.246/event_entity_train_data_label.csv"

In [24]:
train_df = pd.read_csv(train_url, delimiter='\t', header=None)


In [25]:
print(type(train_df[2]))
train_df[2].value_counts()

<class 'pandas.core.series.Series'>


业务资产重组       8000
涉嫌非法集资       4210
股票转让-股权受让    3414
债务违约         3405
涉嫌传销         3273
实控人股东变更      2172
交易违规         2062
不能履职         1573
涉嫌欺诈         1516
涉嫌违法         1421
实际控制人变更      1404
重组失败         1283
业绩下滑          834
财务信息造假        723
提现困难          710
财务造假          705
资金紧张          591
商业信息泄露        554
实际控制人涉诉仲裁     519
歇业停业          458
失联跑路          386
高管负面          380
资产负面          375
资金账户风险        372
债务重组          342
投诉维权          297
履行连带担保责任       88
Name: 2, dtype: int64

In [32]:
model_class, tokenizer_class, pretrained_weights = (tfs.BertModel, tfs.BertTokenizer, 'bert-base-chinese')


In [33]:
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)

In [34]:
model = model_class.from_pretrained(pretrained_weights)


In [35]:
tokenizer.encode("hello world", add_special_tokens=True)

[101, 8701, 8572, 102]

In [36]:
tokenizer.encode("hello world你好啊", add_special_tokens=True)

[101, 8701, 8572, 872, 1962, 1557, 102]

In [40]:
train_set = train_df

In [41]:
train_tokenized = train_set[1].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

Token indices sequence length is longer than the specified maximum sequence length for this model (2732 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (537 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (537 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (7969 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1902 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for 

In [ ]:
train_max_len = 0
for i in train_tokenized.values:
    if len(i) > train_max_len:
        train_max_len = len(i)

train_padded = np.array([i + [0] * (train_max_len-len(i)) for i in train_tokenized.values])
print("train set shape:",train_padded.shape)
